<a href="https://colab.research.google.com/github/chinmay002/Pytorch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import torch
import re,json,string,os
import collections
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [ ]:
n_features =2 
hidden_dim =2
torch.manual_seed(19)
rnn_cell = nn.RNNCell(n_features,hidden_dim)
rnn_state = rnn_cell.state_dict()
rnn_state

#weights ih and bias ih is for inputs vectors
#weights hh and bh is for hidden stat

OrderedDict([('weight_ih',
              tensor([[ 0.6627, -0.4245],
                      [ 0.5373,  0.2294]])),
             ('weight_hh',
              tensor([[-0.4015, -0.5385],
                      [-0.1956, -0.6835]])),
             ('bias_ih', tensor([0.4954, 0.6533])),
             ('bias_hh', tensor([-0.3565, -0.2904]))])

In [ ]:
linear_input = nn.Linear(n_features, hidden_dim)
linear_hidden = nn.Linear(hidden_dim, hidden_dim)

with torch.no_grad():
    linear_input.weight = nn.Parameter(rnn_state['weight_ih'])
    linear_input.bias = nn.Parameter(rnn_state['bias_ih'])
    linear_hidden.weight = nn.Parameter(rnn_state['weight_hh'])
    linear_hidden.bias = nn.Parameter(rnn_state['bias_hh'])

In [ ]:
initial_hidden = torch.zeros(1,hidden_dim)
initial_hidden

tensor([[0., 0.]])

In [ ]:
th = linear_hidden(initial_hidden)
th

tensor([[-0.3565, -0.2904]], grad_fn=<AddmmBackward0>)

In [ ]:
X = torch.as_tensor([[ 1.0349,  0.9661],
        [ 0.8055, -0.9169],
        [-0.8251, -0.9499],
        [-0.8670,  0.9342]])

In [ ]:
tx = linear_input(X[:1])#get the first point 
tx

tensor([[0.7712, 1.4310]], grad_fn=<AddmmBackward0>)

In [ ]:
#Adding tx+th 
torch.tanh(tx+th)
#its an updated hidden state

tensor([[0.3924, 0.8146]], grad_fn=<TanhBackward0>)

In [ ]:
#now lets check this` in rnncell 
rnn_cell(X[:1])

tensor([[0.3924, 0.8146]], grad_fn=<TanhBackward0>)

In [ ]:
from pandas.core.arraylike import default_array_ufunc
rnn_cell(X)
#this is worg!!!!!!
#why?
#remember, rnn_cell has two inputs (inp and hidden). if hidden is not specified each time it 
#will take default as zero 


tensor([[ 0.3924,  0.8146],
        [ 0.7864,  0.5266],
        [-0.0047, -0.2898],
        [-0.6817,  0.1109]], grad_fn=<TanhBackward0>)

In [ ]:
#so we need to loop and provide updated hidden state at each timestep
hidden = torch.zeros(1,hidden_dim)
for i in range(X.shape[0]):
  out = rnn_cell(X[i:i+1],hidden)
  print(out)
  hidden = out

#the last state[[-0.5297,  0.3551]], is representation of full sequences

tensor([[0.3924, 0.8146]], grad_fn=<TanhBackward0>)
tensor([[ 0.4347, -0.0481]], grad_fn=<TanhBackward0>)
tensor([[-0.1521, -0.3368]], grad_fn=<TanhBackward0>)
tensor([[-0.5297,  0.3551]], grad_fn=<TanhBackward0>)


**Shape and sizes of vectors**



**Input_shape**
<li>default :(seq_len, bacth_size, n_of_feat)</li>
<li>batch_first provide in (batch, seq_len, n_feat)</li>

**hidden_state_Shape**
<li>simple rnn hidden_state is (1 ,batch_size, hidd_dim)</li>
<li>stacked RNN (n_stacked_layer, batch_size , hidd_dim)</li>
<li>Bi_direction (2*n_stack_layer, batch_size, hidd_dim)</li>

**Output_SHape**
<li>simple : (seq_lem , batch_size , hidd_dim)</li>
<li>Bi-Direction : (seq_len, batch_size, 2*hidd)</li>
<li>btach_first : (batch_size, seq_len, hidd_dim)</li>

In [ ]:
batch = torch.randn(3, 4, 2)
batch.shape

torch.Size([3, 4, 2])

In [ ]:
#RNN uses seq first, we need to change it to batch first
permute_batch = batch.permute(1,0,2)
permute_batch.shape
#now it is RNN friendly

torch.Size([4, 3, 2])

In [ ]:
rnn_cell = nn.RNN(input_size = 2,hidden_size = 2)
out , final_hidden = rnn_cell(permute_batch) #applies tanh(wx+b)
out.shape,final_hidden.shape



(torch.Size([4, 3, 2]), torch.Size([1, 3, 2]))

In [ ]:
#for simple rnn, last stage output is final output
final_hidden

tensor([[[ 0.3821,  0.5625],
         [ 0.6310, -0.2361],
         [ 0.8145,  0.7109]]], grad_fn=<StackBackward0>)

In [ ]:
out[-1]

tensor([[ 0.3821,  0.5625],
        [ 0.6310, -0.2361],
        [ 0.8145,  0.7109]], grad_fn=<SelectBackward0>)

In [ ]:
(out[-1]==final_hidden).all()

tensor(True)

In [ ]:
#or we can keep batch first True
rnn_batch_first = nn.RNN(2,2,batch_first=True)
out,final_hidden = rnn_batch_first(batch)
out.shape,final_hidden.shape

(torch.Size([3, 4, 2]), torch.Size([1, 3, 2]))

**Stacked RNN**
output of rnn are fed to other rnns.
layer 1 with rnn cell 1
layer 0 with rnn cells 0



In [ ]:
rnn_stacked = nn.RNN(input_size = 2,hidden_size=2,num_layers=2, batch_first=True)
state = rnn_stacked.state_dict()
state
#weights abd biases of layer l_0 and l_1

OrderedDict([('weight_ih_l0',
              tensor([[-0.0316, -0.6799],
                      [-0.6724,  0.0982]])),
             ('weight_hh_l0',
              tensor([[ 0.2065,  0.3822],
                      [ 0.4892, -0.0878]])),
             ('bias_ih_l0', tensor([ 0.0165, -0.6195])),
             ('bias_hh_l0', tensor([0.0245, 0.4649])),
             ('weight_ih_l1',
              tensor([[-0.2977,  0.4305],
                      [-0.0030, -0.5329]])),
             ('weight_hh_l1',
              tensor([[ 0.0258, -0.6403],
                      [ 0.1777,  0.2684]])),
             ('bias_ih_l1', tensor([0.6674, 0.3733])),
             ('bias_hh_l1', tensor([-0.2702,  0.4040]))])

In [ ]:
rnn_layer_0 = nn.RNN(input_size=2,hidden_size=2,batch_first =True)
rnn_layer_1 = nn.RNN(input_size=2,hidden_size=2,batch_first =True)

rnn_layer_0.load_state_dict(dict(list(state.items())[:4]))
rnn_layer_0.load_state_dict(dict([(k[:-1]+'0',v) for k,v in list(state.items())[:4]]))
#keeping the variable name same but different wieghts.

<All keys matched successfully>

In [ ]:
X = torch.randn(1,4,2) #1 batch, 4 seq_len, 2 feat
X = X[:1]
out0,h0 = rnn_layer_0(X)
out1,h1 = rnn_layer_1(out0)


In [ ]:
out1,h0

(tensor([[[ 0.7609,  0.1250],
          [ 0.1387,  0.0074],
          [ 0.6237, -0.1651],
          [ 0.2745, -0.1441]]], grad_fn=<TransposeBackward1>),
 tensor([[[ 0.7399, -0.2148]]], grad_fn=<StackBackward0>))

In [ ]:
out1,h1

(tensor([[[ 0.7609,  0.1250],
          [ 0.1387,  0.0074],
          [ 0.6237, -0.1651],
          [ 0.2745, -0.1441]]], grad_fn=<TransposeBackward1>),
 tensor([[[ 0.2745, -0.1441]]], grad_fn=<StackBackward0>))

In [ ]:
out1,torch.cat([h0,h1])

(tensor([[[ 0.7609,  0.1250],
          [ 0.1387,  0.0074],
          [ 0.6237, -0.1651],
          [ 0.2745, -0.1441]]], grad_fn=<TransposeBackward1>),
 tensor([[[ 0.7399, -0.2148]],
 
         [[ 0.2745, -0.1441]]], grad_fn=<CatBackward0>))

In [ ]:
#cross cheeck
out, hiddn = rnn_stacked(X)
out,hiddn

(tensor([[[ 0.5626,  0.3755],
          [-0.0507,  0.7475],
          [-0.1235,  0.6402],
          [-0.3173,  0.7777]]], grad_fn=<TransposeBackward1>),
 tensor([[[ 0.7399, -0.2148]],
 
         [[-0.3173,  0.7777]]], grad_fn=<StackBackward0>))

In [ ]:
#simple rnn, final output elment is hidden state of last layer.
#since its batch first, we need to make hidden stae dim to batch first as welll
out[:,-1]

tensor([[-0.3173,  0.7777]], grad_fn=<SelectBackward0>)

In [ ]:
hiddn.permute(1,0,2)[:,-1]

tensor([[-0.3173,  0.7777]], grad_fn=<SelectBackward0>)

In [ ]:
for k,v in list(state.items())[4:]:
  print(k[:-1]+'0',v)

weight_ih_l0 tensor([[-0.2977,  0.4305],
        [-0.0030, -0.5329]])
weight_hh_l0 tensor([[ 0.0258, -0.6403],
        [ 0.1777,  0.2684]])
bias_ih_l0 tensor([0.6674, 0.3733])
bias_hh_l0 tensor([-0.2702,  0.4040])


**Bidirectional RNN**
<li>for one RNN we feed data in sequwnce of points and for another data we feed data in reverese order</li>

<li>Unlike stacked RNN which uses its hidden states  as inpu to next RNN, bidrectiona; keeps hidden states of both layer </li>

In [ ]:
torch.manual_seed(19)
rnn_bidirect = nn.RNN(input_size=2,hidden_size = 2 , bidirectional= True , batch_first = True)
state = rnn_bidirect.state_dict()
state

OrderedDict([('weight_ih_l0',
              tensor([[ 0.6627, -0.4245],
                      [ 0.5373,  0.2294]])),
             ('weight_hh_l0',
              tensor([[-0.4015, -0.5385],
                      [-0.1956, -0.6835]])),
             ('bias_ih_l0', tensor([0.4954, 0.6533])),
             ('bias_hh_l0', tensor([-0.3565, -0.2904])),
             ('weight_ih_l0_reverse',
              tensor([[-0.6701, -0.5811],
                      [-0.0170, -0.5856]])),
             ('weight_hh_l0_reverse',
              tensor([[ 0.1159, -0.6978],
                      [ 0.3241, -0.0983]])),
             ('bias_ih_l0_reverse', tensor([-0.3163, -0.2153])),
             ('bias_hh_l0_reverse', tensor([ 0.0722, -0.3242]))])

In [ ]:
for k, v in list(state.items())[4:]:
  print(k[:-8])

weight_ih_l0
weight_hh_l0
bias_ih_l0
bias_hh_l0


In [ ]:
rnn_forward = nn.RNN(input_size=2,hidden_size=2,batch_first = True)
rnn_reverse = nn.RNN(input_size = 2,hidden_size = 2, batch_first=True)

rnn_forward.load_state_dict(dict(list(state.items())[:4]))
rnn_reverse.load_state_dict( dict([(k[:-8],v)  for k, v in list(state.items())[4:]]))

<All keys matched successfully>

In [ ]:
X = torch.randn(1,4,2)


In [ ]:
X_rev = torch.flip(X,dims=[1])  #reverses the order of inputs

In [ ]:
out,h = rnn_forward(X)
out_rev,h_rev = rnn_reverse(X_rev)


In [ ]:
print(out.shape,h.shape)
out

torch.Size([1, 4, 2]) torch.Size([1, 1, 2])


tensor([[[ 0.6383,  0.3747],
         [ 0.2004,  0.8140],
         [-0.2374, -0.7248],
         [ 0.7238,  0.8588]]], grad_fn=<TransposeBackward1>)

In [ ]:
print(out_rev.shape,h_rev.shape)
out_rev

torch.Size([1, 4, 2]) torch.Size([1, 1, 2])


tensor([[[-0.6636, -0.6050],
         [ 0.8776,  0.1293],
         [-0.9676, -0.7794],
         [ 0.3715, -0.2956]]], grad_fn=<TransposeBackward1>)

In [ ]:
out_rev_back = torch.flip(out_rev,dims =[1])

In [ ]:
torch.cat([out,out_rev_back],dim=2),torch.cat([h,h_rev])

(tensor([[[ 0.6383,  0.3747,  0.3715, -0.2956],
          [ 0.2004,  0.8140, -0.9676, -0.7794],
          [-0.2374, -0.7248,  0.8776,  0.1293],
          [ 0.7238,  0.8588, -0.6636, -0.6050]]], grad_fn=<CatBackward0>),
 tensor([[[ 0.7238,  0.8588]],
 
         [[ 0.3715, -0.2956]]], grad_fn=<CatBackward0>))

In [ ]:
out,hidden = rnn_bidirect(X)
out,hidden 

(tensor([[[ 0.6383,  0.3747,  0.3715, -0.2956],
          [ 0.2004,  0.8140, -0.9676, -0.7794],
          [-0.2374, -0.7248,  0.8776,  0.1293],
          [ 0.7238,  0.8588, -0.6636, -0.6050]]], grad_fn=<TransposeBackward1>),
 tensor([[[ 0.7238,  0.8588]],
 
         [[ 0.3715, -0.2956]]], grad_fn=<StackBackward0>))

In [ ]:
#in BIdredt last state of output is not final hidden state
#make hidden state dim to batch_first

hidden.permute(1,0,2).view(1,-1)

tensor([[ 0.7238,  0.8588,  0.3715, -0.2956]], grad_fn=<ViewBackward0>)

**GRU**

In [4]:
n_features =2
n_dim = 2
torch.manual_seed(17)
gru_cell = nn.GRUCell(input_size=n_features,hidden_size = n_dim)
gru_state = gru_cell.state_dict()
gru_state

#n,z,k are all concatenated in Weight matirx and each of them have sixe of n_dim

OrderedDict([('weight_ih',
              tensor([[-0.0930,  0.0497],
                      [ 0.4670, -0.5319],
                      [-0.6656,  0.0699],
                      [-0.1662,  0.0654],
                      [-0.0449, -0.6828],
                      [-0.6769, -0.1889]])),
             ('weight_hh',
              tensor([[-0.4167, -0.4352],
                      [-0.2060, -0.3989],
                      [-0.7070, -0.5083],
                      [ 0.1418,  0.0930],
                      [-0.5729, -0.5700],
                      [-0.1818, -0.6691]])),
             ('bias_ih',
              tensor([-0.4316,  0.4019,  0.1222, -0.4647, -0.5578,  0.4493])),
             ('bias_hh',
              tensor([-0.6800,  0.4422, -0.3559, -0.0279,  0.6553,  0.2918]))])

In [7]:
Wx,bx = gru_state['weight_ih'],gru_state['bias_ih']
Wh,bh = gru_state['weight_hh'],gru_state['bias_hh']

In [10]:
print(f'input shape {Wx.shape} {bx.shape}')
print(f'input shape {Wh.shape} {bh.shape}')

input shape torch.Size([6, 2]) torch.Size([6])
input shape torch.Size([6, 2]) torch.Size([6])


In [18]:
Wxr,Wxz,Wxn = Wx.split(2)
bxr,bxz,bxn = bx.split(2)

Whr,Whz,Whn = Wh.split(2)
bhr,bhz,bhn = bh.split(2)

In [15]:
Wx.size()

torch.Size([6, 2])

In [24]:
def linear_layer(Wx,bx,Wh,bh):
  hidden_dim , n_feat = Wx.size()#(2,2)
  lin_input = nn.Linear(n_feat,hidden_dim)
  lin_input.load_state_dict({'weight':Wx,'bias':bx})
  lin_hidden = nn.Linear(hidden_dim,hidden_dim)
  lin_hidden.load_state_dict({'weight':Wh,'bias':bh})

  return lin_hidden,lin_input

In [26]:
r_hidden,r_input = linear_layer(Wxr,bxr,Whr,bhr)
z_hidden,z_input = linear_layer(Wxz,bxz,Whz,bhz)
n_hidden,n_input = linear_layer(Wxn,bxn,Whn,bhn)


In [27]:
def reset_gate(h,x):
  thr = r_hidden(x)
  txr = r_input(x)
  r = torch.sigmoid(thr+txr)
  return r

def update_gate(h,x):
  thz = z_hidden(x)
  txz = z_input(x)
  z = torch.sigmoid(thz+txz)
  return z

def candidate_n(h,x,r):
  thn = n_hidden(x)
  txn = n_input(x)
  n = torch.sigmoid( r * thn + txn)
  return n    

In [30]:
initial_hidden_state = torch.zeros(1,n_dim)
X = torch.as_tensor([[ 1.0349,  0.9661]])
r = reset_gate(initial_hidden_state,X)
r

tensor([[0.1180, 0.5535]], grad_fn=<SigmoidBackward0>)

In [31]:
#rest_gate is input ot candidtae 
n = candidate_n(initial_hidden_state,X,r)
n

tensor([[0.2106, 0.3243]], grad_fn=<SigmoidBackward0>)

In [34]:
z =update_gate(initial_hidden_state,X)
z
#update gate is telling to keep 11% and 41% of in initla_hidden_stae and remaining is coming from candidiate

tensor([[0.1113, 0.4098]], grad_fn=<SigmoidBackward0>)

In [35]:
h_prime = n*(1-z)+initial_hidden_state*z
h_prime

tensor([[0.1871, 0.1914]], grad_fn=<AddBackward0>)

In [36]:
gru_cell(X)

tensor([[-0.5635, -0.1470]], grad_fn=<AddBackward0>)

In [ ]:
class ElmanRNN(nn.Module):

  def __intit__(self,input_size,hidden_size,batch_first = False):
    super(ElmanRNN,self).__init__()
    self.rnn_cell = nn.RNNCell(input_size=input_size,hidden_size = hidden_size)

    self.batch_first = batch_first
    self.hidden_size = hidden_size

  def _initialize_hidden(self,batch_size)  :
    return torch.zeros((batch_size,self.hidden_size))

  def forward(self,x_in,initial_hidden = None):
    '''
    Args :
    x_in : if self.batch_first:
          x_in.shape will be (batch, seq_size, feat_size)
     else:         X_in.shape (seq_size,batch_size,feat_size)
     Return:
     hiiden vectors; output of RNN at each timestep
       if self.bacth_first : shape will be (batch_size, seq_size, hidden_size)
                      else, hidden.shape = (seq_size,batch_size,hidden_size)            
    '''
    if self.batch_first:
      batch_size ,seq_size,feat_size = x_in.shape()
      x_in = x_in.permute(1,0,2)
    else:
      seq_size,batch_size,feat_size = x_in.size()

    hiddens = []

    if initial_hidden is None:  
      initial_hidden = self._initialize_hidden(batch_size)
      initial_hidden = initial_hidden.to(x_in.device)

    for t in range(seq_size)  :
      hidden_t = self.rnn_cell(x_in[t],hidden_t)
      hiddens.append(hidden_t)

    hiddens = torch.stack(hiddens) 

    if self.batch_first :
      hidden = hiddens.permute(1,0,2)

    return hiddens   

In [ ]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        """
        Args:
            surname_df (pandas.DataFrame): the dataset
            vectorizer (SurnameVectorizer): vectorizer instatiated from dataset
        """
        self.surname_df = surname_df
        self._vectorizer = vectorizer

        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
        # Class weights
        class_counts = surname_df.nationality.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.nationality_vocab.lookup_token(item[0])
        
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            surname_csv (str): location of the dataset
        Returns:
            an instance of SurnameDataset
        """
        surname_df = pd.read_csv(surname_csv)
        train_surname_df = surname_df[surname_df.split=='train']
        return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            surname_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of SurnameDataset
        """
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of SurnameVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's:
                features (x_surname)
                label (y_nationality)
        """
        row = self._target_df.iloc[index]

        surname_vector = self._vectorizer.vectorize(row.surname) #this  goes to SurVect Class
        print(f'vectorzied surname of {row.surname} --> {surname_vector}')

        nationality_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_surname': surname_vector,
                'y_nationality': nationality_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [ ]:
class Vocabulary(object):
  
  def __init__(self,token_to_idx=None):
    
    if token_to_idx is None:
      token_to_idx = {}

    self._token_to_idx = token_to_idx
    self._idx_to_token = {v:k for k,v in self._token_to_idx.items()}
    
    # self._add_unk = add_unk
    # self._unk_token = unk_token
    # self._mask_token = mask_token
    
    # self._mask_index = self.add_token(self._mask_token)

    # self.unk_index = -1
    # if add_unk:
    #   self.unk_index = self.add_token(unk_token)
    

  def add_token(self,token):
    '''
      add a token to dic torkn_to_idx

    '''
    if token in self._token_to_idx:
      index = self._token_to_idx[token]

    else:
      index = len(self._token_to_idx) 
      self._idx_to_token[index] = token
      self._token_to_idx[token] = index 

    return index

  def lookup_token(self,token):
    '''
    get the index of the token
    else, get the unk index if token not present
    '''  
    # if self._unk_index >=0:
    #   return self._token_to_idx.get(token,self._unk_index)
    # else:
    return self._token_to_idx[token]  
 
  def lookup_index(self, index):
    """Return the token associated with the index
    
    Args: 
        index (int): the index to look up
    Returns:
        token (str): the token corresponding to the index
    Raises:
        KeyError: if the index is not in the Vocabulary
    """
    if index not in self._idx_to_token:
        raise KeyError("the index (%d) is not in the Vocabulary" % index)
    return self._idx_to_token[index]   \

  def __len__(self):
    return len(self._token_to_idx)
  
  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)

  def to_serializable(self):
    """ returns a dictionary that can be serialized """
    return {'token_to_idx': self._token_to_idx}

  @classmethod
  def from_serializable(cls, contents):
    """ instantiates the Vocabulary from a serialized dictionary """
    return cls(**contents)  

    

In [ ]:
class SequenceVocabulary(Vocabulary):
  def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

    
    super(SequenceVocabulary, self).__init__(token_to_idx) #super retunr obj of calss vocabulary
    #and Vocab class is called with token_to_idx param

    self._mask_token = mask_token  #its padding 
    self._unk_token = unk_token
    self._begin_seq_token = begin_seq_token
    self._end_seq_token = end_seq_token

    self.mask_index = self.add_token(self._mask_token)
    self.unk_index = self.add_token(self._unk_token)
    self.begin_seq_index = self.add_token(self._begin_seq_token)
    self.end_seq_index = self.add_token(self._end_seq_token)

  def to_serializable(self):
    contents = super(SequenceVocabulary, self).to_serializable()
    contents.update({'unk_token': self._unk_token,
                      'mask_token': self._mask_token,
                      'begin_seq_token': self._begin_seq_token,
                      'end_seq_token': self._end_seq_token})
    return contents

  def lookup_token(self, token):
    """Retrieve the index associated with the token 
      or the UNK index if token isn't present.
    
    Args:
        token (str): the token to look up 
    Returns:
        index (int): the index corresponding to the token
    Notes:
        `unk_index` needs to be >=0 (having been added into the Vocabulary) 
          for the UNK functionality 
    """
    if self.unk_index >= 0:
        return self._token_to_idx.get(token, self.unk_index)
    else:
        return self._token_to_idx[token]


In [ ]:
class SurnameVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, surname_vocab, nationality_vocab):
        """
        Args:
            surname_vocab (Vocabulary): maps characters to integers
            nationality_vocab (Vocabulary): maps nationalities to integers
        """
        self.surname_vocab = surname_vocab
        self.nationality_vocab = nationality_vocab
        print(f'surname_vocab length {len(self.surname_vocab)}')
        print(f'nationality_vocab length {len(self.nationality_vocab)}')

    def vectorize(self, surname):
        """
        Args:
            surname (str): the surname

        Returns:
            one_hot (np.ndarray): a collapsed one-hot encoding 
        """
        vocab = self.surname_vocab    #{....} 80 key value pair
        one_hot = np.zeros(len(vocab), dtype=np.float32) #len is 80 [0,0,0,0,....]
        for token in surname:  # Macfauuen
            print(f'printitng vectorize token {token}')
            one_hot[vocab.lookup_token(token)] = 1

        return one_hot #returned to DataSet class

    @classmethod
    def from_dataframe(cls, surname_df):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            surname_df (pandas.DataFrame): the surnames dataset
        Returns:
            an instance of the SurnameVectorizer
        """
        surname_vocab = Vocabulary(unk_token="@")
        nationality_vocab = Vocabulary(add_unk=False)

        for index, row in surname_df.iterrows():
            print(f'printing index, row in {row.surname} {row.nationality}')
            for letter in row.surname:
                surname_vocab.add_token(letter)
            nationality_vocab.add_token(row.nationality)

        return cls(surname_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        surname_vocab = Vocabulary.from_serializable(contents['surname_vocab'])
        nationality_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])
        return cls(surname_vocab=surname_vocab, nationality_vocab=nationality_vocab)

    def to_serializable(self):
        return {'surname_vocab': self.surname_vocab.to_serializable(),
                'nationality_vocab': self.nationality_vocab.to_serializable()}

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
from argparse import Namespace
args = Namespace(
    #@ Data and path Information:
    surname_csv = 'surname_splits.csv',
    vectorizer_file = "vectorizer.json",
    model_state_file = "model.pth",
    save_dir = "model_storage/surname_mlp",
    #@ Model Hyperparameters:
    hidden_dim = 100,
    #@ Training Hyperparameters:
    seed = 42,
    num_epochs = 5,
    early_stopping_criteria = 5,
    learning_rate = 0.001,
    batch_size = 128,
    #@ Runtime Options:
    cuda = True,
    reload_from_files = False,
    expand_filepaths_to_save_dir = True
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)
    print("Expanded Filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

#@ Checking the CUDA:
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

#@ Set seed for Reproducibility:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
set_seed_everywhere(args.seed, args.cuda)


#@ Handle dirs:
def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
handle_dirs(args.save_dir)


In [ ]:
if args.reload_from_files:
    # training from a checkpoint
    print("Reloading!")
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
else:
    # create dataset and vectorizer
    print("Creating fresh!")
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)
    
vectorizer = dataset.get_vectorizer()
classifier = SurnameClassifier(input_dim = len(vectorizer.surname_vocab),   #80
                               hid_dim = args.hidden_dim,                   #300
                               out_dim= len(vectorizer.nationality_vocab))  #18

#our vocab has 80 uniqye character with their index in dic form and nationality has 18 unique 

In [ ]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate, weight_decay= 0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

import time as t
t1 = t.perf_counter()

try:
    for epoch_index in range(args.num_epochs):

        # Record the tarining stage
        if (epoch_index+1) % 10 == 0:
            print("Epoch [{}/{}]".format(epoch_index+1, args.num_epochs))
        
        train_state['epoch_index'] = epoch_index
        
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_surname'])  #goes to get item and picks
            print(f'y_pred {y_pred.shape}')
            print(batch_dict['y_nationality'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            print(f'loss {loss}')
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)


        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_surname'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            loss_t = loss.to("cpu").item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        

except KeyboardInterrupt:
    print("Exiting loop")

print('-----')
t2 = t.perf_counter()
print("Running time = {:.2f}".format(t2-t1))

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_surname'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_nationality'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
def predict_nationality(surname, classifier, vectorizer):
    """Predict the nationality from a new surname
    
    Args:
        surname (str): the surname to classifier
        classifier (SurnameClassifer): an instance of the classifier
        vectorizer (SurnameVectorizer): the corresponding vectorizer
    Returns:
        a dictionary with the most likely nationality and its probability
    """
    vectorized_surname = vectorizer.vectorize(surname)
    print(f' vectorized surname {vectorized_surname}')
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(0)
    print(vectorized_surname)
    result = classifier(vectorized_surname, apply_softmax=True)
    print(result.max(dim=1))
    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality, 'probability': probability_value}

In [ ]:
b